# Data Cleaning

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py


In [2]:
conn = sqlite3.Connection("./capstone.sqlite")

df = pd.read_sql("SELECT * FROM open_pv_clean", con=conn)

DatabaseError: Execution failed on sql 'SELECT * FROM open_pv_clean': no such table: open_pv_clean

In [ ]:
# maybe we can impute cost? probably a function of technology and size.

# cost is an object, let's change that.

df['cost'].value_counts()

In [ ]:
df['cost'] = pd.to_numeric(df['cost'])

In [ ]:
df['tech_1'].value_counts()

In [ ]:
# looks like size and kw are very highly correlated.

df[['cost', 'size_kw', 'annual_PV_prod']].corr()

In [ ]:
df['install_type'].value_counts()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
cost_size = df[['cost', 'size_kw']]
cost_size = cost_size.dropna()

In [ ]:
lin_reg = LinearRegression()

In [ ]:
cost_size.head()

In [ ]:
X = cost_size['size_kw'].values.reshape((len(cost_size['size_kw']), 1))

lin_reg.fit(X=X, y=np.array(cost_size['cost']))

In [ ]:
def fill_cost(row):
    if np.isnan(row['cost']) and not np.isnan(row['size_kw']):
        row['cost'] = lin_reg.predict(row['size_kw'])[0]
    return row


In [ ]:
df = df.apply(fill_cost, axis=1)

In [ ]:
df['cost'].isnull().sum()

In [ ]:
df.to_sql("open_pv_clean_backup", con=conn, if_exists="replace")

In [ ]:
df.to_sql("open_pv_clean", con=conn, if_exists="replace")

# Moving to some Columns with Few Values

In [ ]:
df = pd.read_sql("SELECT * FROM open_pv_clean", con=conn)

In [ ]:
df.info()

In [ ]:
# year and date installed are just all the same. ditching year.

df.loc[df.year.notnull(), ['year', 'date_installed']]

In [ ]:
del(df['year'])

In [ ]:
# there's just too little information here to do anthing.

df['pbi_length'].value_counts()

In [ ]:
del(df['pbi_length'])

In [ ]:
df.loc[df.utility.notnull(), 'utility_clean'] = df.loc[df.utility.notnull(), 'utility']

In [ ]:
del[df['utility']]

In [ ]:
# let's just make a new column for this, it's for buildings, just 1 or 0 if they have it.

df.loc[df['bipv_1'].notnull(), ['bipv_1', 'bipv_2', 'bipv_3']]

In [ ]:
df['bipv'] = [1 if bipv == 1.0 else 0 for bipv in df['bipv_1']]

In [ ]:
del(df['bipv_1'])
del(df['bipv_2'])
del(df['bipv_3'])

In [ ]:
# I guess this is whether it's a new construction or somehting? i'll just fill na with 0

df['new_constr'].value_counts()

In [ ]:
df['new_constr'].fillna(0, inplace=True)

In [ ]:
# this probably corresponds to if these are "tracking" solar panels, we'll assume the ones that are blank are not,
# so just fill with 0

df['tracking'].value_counts()

In [ ]:
df['tracking'].fillna(0, inplace=True)

In [ ]:
# I guess this is whether it's a new construction or somehting? i'll just fill na with 0

df['new_constr'].value_counts()

In [ ]:
df['new_constr'].fillna(0, inplace=True)

In [ ]:
# this probably corresponds to if these are "tracking" solar panels, we'll assume the ones that are blank are not,
# so just fill with 0

df['tracking'].value_counts()

In [ ]:
df['tracking'].fillna(0, inplace=True)

In [ ]:
df.loc[df['manuf1_clean'].notnull(), ['manuf1_clean', 'manuf2_clean', 'manuf3_clean']].head(100)

In [ ]:
del(df['manuf1_clean'])
del(df['manuf3_clean'])

In [ ]:
df.rename(columns={'manuf2_clean': "manufacturer"}, inplace=True)

In [ ]:
df.info()

In [ ]:
# we may be able to fill in some manufacturer values, i wouldn't be surprised if it's really connected to
# installer,

# also may be connected to size and cost?

df.to_sql("open_pv_clean_backup", con=conn, if_exists='replace')

In [ ]:
df.to_sql('open_pv_clean', con=conn, if_exists='replace')